In [15]:
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
import pandas as pd
%run rookie_scraper.ipynb

rookie_scraper.ipynb:4: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 4 of the file rookie_scraper.ipynb. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  "cell_type": "code",
rookie_scraper.ipynb:4: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 4 of the file rookie_scraper.ipynb. To get rid of this warning, pass the additional argument 'features="lxml"' to the B

##Scrape 
Get per game and advanced stats for each year and join on the columns (Player, Pos, Age, Tm, G) as these are common 

In [2]:
def get_stats(year, statstype):
    url = "https://www.basketball-reference.com/leagues/NBA_{}_{}.html".format(year,statstype)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    #soup.findAll('tr', limit=2)

    headers = [th.getText() for th in soup.findAll('tr',limit=2)[0].findAll('th')]
    headers = headers[1:]

    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    return stats.drop_duplicates().dropna(how='all')


len(get_stats(2019,'per_game'))


708

In [3]:
def get_joint_stats(year):
    per_game = get_stats(year,'per_game')
    advanced = get_stats(year,'advanced')
    joint = pd.merge(per_game,advanced, on=['Player', 'Pos', 'Age', 'Tm', 'G'], how='inner', suffixes=('_per_game','_advanced'))
    return joint

len(get_joint_stats(2019))

708

In [4]:
#All stats for all years
initial = get_joint_stats(1984)
for year in range (1985,2020):
    joint = pd.merge(initial, get_joint_stats(year), on='Player', how='outer', suffixes=('','_year'))

In [25]:
d_frames = [get_joint_stats(year) for year in range(1984,2020)]

In [26]:
joint_stats = pd.concat(d_frames,axis=1)

In [27]:
type(names_of_roy_votes)

numpy.ndarray

In [28]:
joint_stats.loc[join_stats['Player'].isin(names_of_roy_votes)]

ValueError: Cannot index with multidimensional key